In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark2 =SparkSession.builder \
  .master('local[*]') \
  .appName('conversions') \
  .config('spark.jars.packages', 'com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.23.2') \
  .config('spark.executor.memory', '4G') \
  .config('spark.driver.memory', '45G') \
  .config('spark.driver.maxResultSize', '10G') \
  .config("spark.memory.offHeap.size","16g") \
  .getOrCreate() 
#   .config("spark.executor.memory", "70g")
#   .config("spark.memory.offHeap.size","8g")  \
#   .config("spark.driver.memory", "8g") \
sqlContext = SQLContext(spark2)

In [2]:
df = sqlContext.read.format("csv").option("header","true").option("delimiter", ",").load("gs://nft_csvingestion/transfers.csv")
df.show(10)

+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------+
|             eventId|    transaction_hash|block_number|         nft_address|            token_id|        from_address|          to_address|transaction_value| timestamp|
+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------+
|cd816651-56b2-4ed...|0x0c73daad5e6946e...|    12936373|0x629A673A8242c2A...|66829341423052781...|0x25f1d709b329C73...|0xC69b4c6fFDBaF84...|                0|1627776481|
|82cc5228-eb80-4e0...|0x0c73daad5e6946e...|    12936373|0x629A673A8242c2A...|11985074437117418...|0x0737E7162C88E9F...|0xC69b4c6fFDBaF84...|                0|1627776481|
|6e1f9cc4-d1df-4a6...|0x0c73daad5e6946e...|    12936373|0x629A673A8242c2A...|15412203442814369...|0xA0fEc186c4b3FAd...|0xC69b4c6fFDBaF84...|          

In [3]:
df.count()

4514729

In [10]:
df.printSchema()

root
 |-- eventId: string (nullable = true)
 |-- transaction_hash: string (nullable = true)
 |-- block_number: string (nullable = true)
 |-- nft_address: string (nullable = true)
 |-- token_id: string (nullable = true)
 |-- from_address: string (nullable = true)
 |-- to_address: string (nullable = true)
 |-- transaction_value: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [11]:
df.write.format('bigquery').option("temporaryGcsBucket","temp_bucket_temp").option('table', 'ExtendedSqlLiteDataset.transfers').partitionBy("nft_address").mode('overwrite').save()

Py4JJavaError: An error occurred while calling o110.save.
: java.lang.ClassNotFoundException: Failed to find data source: bigquery. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:692)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:746)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:993)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:311)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:301)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at